# Projeto #2 - Classificador supervisionado

Antes de começar, leia as [Instruções](https://github.com/thvmm/pos-ds-ia/tree/master/projeto_2#instru%C3%A7%C3%B5es) e os [Critérios de Avaliação](https://github.com/thvmm/pos-ds-ia/tree/master/projeto_2#crit%C3%A9rios-de-avalia%C3%A7%C3%A3o)


### 1) Qual a base escolhida?

O dataset consiste em 20 grupos de notícias, que compreende cerca de 18.000 postagens de notícias, sobre 20 tópicos divididos em dois subconjuntos: um para treinamento e outro para teste. A divisão entre o treino e o conjunto de teste é baseada em mensagens postadas antes e depois de uma data específica.

Características do dataset:

Features   | Valor
--------- | ------
Classes | 20
Amostras | 18846
Dimensionalidade | 1
Recursos | texto

Link da base: https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html



### 2) **(10%)** Pré-processamento: entendimento do conjunto de dados
- Quais são minhas features?
- Quais são minhas classes?
- Como estão distribuidas minhas classes?
- Checagem se os valores estão dentro de um limite permitido ou razoável.
- Tratamento de valores ausentes por eliminação ou substituição.
- Conversão do tipo de dados.


In [22]:
# Implemente sua análise aqui. Use mais blocos se achar que ficará mais organizado.

from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

from pprint import pprint
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [33]:
print(newsgroups_train.filenames.shape)
print(newsgroups_train.target.shape)
newsgroups_train.target[:10]

(2034,)
(2034,)


array([1, 3, 2, 0, 2, 0, 2, 1, 2, 1])

In [24]:
cats = ['alt.atheism', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train', categories=cats)

print(list(newsgroups_train.target_names))
print(newsgroups_train.filenames.shape)
print(newsgroups_train.target.shape)
print(newsgroups_train.target[:10])


['alt.atheism', 'sci.space']
(1073,)
(1073,)
[0 1 1 1 0 1 1 0 0 0]


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
categories = ['alt.atheism', 'talk.religion.misc','comp.graphics', 'sci.space']

newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)
vectors.shape

(2034, 34118)

In [26]:
vectors.nnz / float(vectors.shape[0])

159.0132743362832

### 3) **(80%)** Nos blocos seguintes implemente seus classificadores (serão implementados 2 métodos diferentes).

#### 3.1) Qual método escolhido?

**Indique o método escolhido**

Multinomial Naive Bayes

#### 3.2) **(10%)** Baseline - Implemente seu classificador da forma mais simples possível para esse ser seu baseline

In [30]:
# Implementação. Use mais blocos se achar que ficará mais organizado.

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

newsgroups_test = fetch_20newsgroups(subset='test',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
vectors_test = vectorizer.transform(newsgroups_test.data)
pred = clf.predict(vectors_test)
metrics.f1_score(pred, newsgroups_test.target, average='macro')

0.7731035068127478

#### 3.3) **(20%)** Versão 1 - O que podemos fazer para melhorar nosso baseline? Aplique técnicas como redução de dimensionalidade, normalização ou outras. Compare os resultados.

In [31]:
# Implementação. Use mais blocos se achar que ficará mais organizado.
newsgroups_train = fetch_20newsgroups(subset='train',
                                       remove=('headers', 'footers', 'quotes'),
                                       categories=categories)
vectors = vectorizer.fit_transform(newsgroups_train.data)
clf = MultinomialNB(alpha=.01)
clf.fit(vectors, newsgroups_train.target)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [32]:
vectors_test = vectorizer.transform(newsgroups_test.data)
pred = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred, average='macro')

0.7699517518452172

#### 3.4) **(10%)** Tunning - Agora que temos um resultado promissor, vamos tentar melhorar o resultado alterando um ou mais hiper-parametro. Compare os resultados.

In [28]:
# Implementação. Use mais blocos se achar que ficará mais organizado.

newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)

vectors_test = vectorizer.transform(newsgroups_test.data)
clf = MultinomialNB(alpha=.01)
clf.fit(vectors, newsgroups_train.target)

MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True)

In [29]:
pred = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred, average='macro')

0.8821359240272957

#### 3.5) Qual método escolhido?

**Indique o método escolhido**

Multiclass hinge loss with elastic net regularization

#### 3.6) **(10%)** Baseline - Implemente seu classificador da forma mais simples possível para esse ser seu baseline

In [0]:
# Implementação. Use mais blocos se achar que ficará mais organizado.

import numpy as np
import cvxpy as cp
import epopt as ep

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")

#### 3.7) **(20%)** Versão 1 - O que podemos fazer para melhorar nosso baseline? Aplique técnicas como redução de dimensionalidade, normalização ou outras. Compare os resultados.

In [0]:
# Implementação. Use mais blocos se achar que ficará mais organizado.

from sklearn.feature_extraction import text

vectorizer = text.TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(newsgroups_train.data)
y = newsgroups_train.target
Xtest = vectorizer.transform(newsgroups_test.data)
ytest = newsgroups_test.target

print (X.shape)

#### 3.8) **(10%)** Tunning - Agora que temos um resultado promissor, vamos tentar melhorar o resultado alterando um ou mais hiper-parametro. Compare os resultados.

In [0]:
# Implementação. Use mais blocos se achar que ficará mais organizado.

def multiclass_hinge_loss(Theta, X, y):
    k = Theta.size[1]
    Y = one_hot(y, k)
    return (cp.sum_entries(cp.max_entries(X*Theta + 1 - Y, axis=1)) -
            cp.sum_entries(cp.mul_elemwise(X.T.dot(Y), Theta)))

In [0]:
m, n = X.shape
k = '20'
Theta = cp.Variable(n, k)
lam1 = 0.1
lam2 = 1

f = ep.multiclass_hinge_loss(Theta, X, y) + lam1*cp.norm1(Theta) + lam2*cp.sum_squares(Theta)
prob = cp.Problem(cp.Minimize(f))
ep.solve(prob, verbose=True)

Theta0 = np.array(Theta.value)
print ("Train accuracy:", accuracy(np.argmax(X.dot(Theta0), axis=1), y))
print ("Test accuracy:", accuracy(np.argmax(Xtest.dot(Theta0), axis=1), ytest))

### 5) **(10%)** Conclusões

*Compare seus resultados. Imaginando que sua solução fosse para produção, qual deles você escolheria? Por que? Quais os riscos você enxerga? O que recomendaria de próximos passos para melhorar os resultados?*

No primeiro classificador ele se adequou de forma facil a coisas específicas que aparecem no dataset, como os cabeçalhos. Este classificador alcança uma acuracia muito alta, no entanto seus resultados não se generalizam para outros documentos que não estejam no mesmo espaco de tempo.

Ja no segundo classificador foi utilizado uma abordagem direta de geração de recursos e um modelo simples de palavras-chave, foi atingido mais ou menos 80% de precisão. 

Nota-se que para esse dataset, a divisão por data tende a resultar em um erro de generalização menor que o esperado, pois previsivelmente devido ao fato de que o conteúdo de um grupo de notícias específico é mudado ao longo do tempo.

Por tanto, sem dúvida, podemos melhorar esse resultado incluindo, por exemplo, higher order n-grams, colocando recursos de PNL mais sofisticados e várias outras abordagens para a engenharia de fetuares.

